Developement of code for the card game skyjo.

In [1]:
import numpy as np
import random as random
#using simpleguitk for display, is not needed for computer game
import simpleguitk as simplegui
import time

First, I create the class Cards ince the game is played with cards. For the game matters only the number and whether they are open or not. For display also the color and the position is needed. 

In [2]:
#purely for display color with dictionary
dict_col={-2:'blue',-1:'blue',0:'cyan',1:'green',2:'green',3:'green',4:'green',5:'yellow',6:'yellow',7:'yellow',8:'yellow',9:'red',10:'red',11:'red',12:'red'}

In [3]:
#cards of the game
class Card:
    #initiate cards, only number needed since position and state comes mainly from the other Classes
    def __init__(self, number):
        self.number = number        
        self.color = dict_col[number]
        #open or closed, default is closed
        self.open =False
        #default position is None
        self.position=[None,None]  
    #print output    
    def __str__(self):
        if self.open==False:
            return "Card has Value "+str(self.number)+" and is closed."+ \
                " Its position is at "+str(self.position)
        if self.open==True:
            return "Card has Value "+str(self.number)+" and is open."+ \
                " Its position is at "+str(self.position)
    #set card to a state
    def set_state(self,state):
        self.open=state
    #set card to number, mainly for testing    
    def set_number(self,number):
        self.number=number
        self.color = dict_col[number]
    #set position for display  
    def set_position(self,posinput):
        self.position=posinput
    #draw on GUi    
    def draw(self,canvas):
        card_size=[70,50]
        corners=[[self.position],[self.position[0]+card_size[0],self.position[1]],[self.position[0]+card_size[0],self.position[1]+card_size[1]],[self.position[0],self.position[1]+card_size[1]]]
        if self.open==False:
            #closed black
            canvas.draw_polygon(corners,1,'Black','Black')
        else:
            #open colored number on gray
            canvas.draw_polygon(corners,1,'Black','Light Gray')
            centerb=list(self.position)
            centerb[1]+=58
            centerb[0]+=5
            canvas.draw_text(self.number,centerb,50,self.color)

Below I test the class and its methods. 

In [4]:
card1=Card(-2)
print(card1.color)
print(card1.open)
ar=np.zeros((2))
ar[0]=card1.number
ar[1]=card1.open
print(ar)
print(card1)
card1.set_state(True)
print(card1)
card1.set_state(False)
print(card1)

blue
False
[-2.  0.]
Card has Value -2 and is closed. Its position is at [None, None]
Card has Value -2 and is open. Its position is at [None, None]
Card has Value -2 and is closed. Its position is at [None, None]


Pile class which creates Pile (lists) can be created from scratch or from another pile.

In [5]:
#pile of cards can be open or closed, only top most accessable
class Pile:
    #create parameter: string of mode, where to take cards for it, can be None
    def __init__(self,mode,take_from):
        #create new close pile, shuffles it also 
        if mode=='create_closed':
            #all closed in this mode
            self.open=False
            #postion in gui
            self.position=[100,10]
            #number of cards per type
            dict_num={-2:5,-1:10,0:15,1:10,2:10,3:10,4:10,5:10,6:10,7:10,8:10,9:10,10:10,11:10,12:10}
            #create empty list
            self.list_cards=[]
            #add as many cards as exist of each type
            for i in range(15):
                for j in range(dict_num[i-2]):
                    #create card
                    cardin=Card(i-2)
                    #use position of Pile
                    cardin.set_position(self.position)
                    #and state of Pile
                    cardin.set_state(self.open)
                    #append
                    self.list_cards.append(cardin)    
                    #print content 
                    self.list_cards
            #shuffle cards        
            random.shuffle(self.list_cards)     
        #create open pile from other    starts with just one card
        if mode=='create_open': 
            #postion in gui
            self.position=[300,10]   
            #is open 
            self.open=True
            self.list_cards=[]
            self.list_cards.append(take_from.list_cards.pop(-1))
            #set card position properties to Pile properties
            self.list_cards[0].set_state(self.open)
            self.list_cards[0].set_position(self.position)            
    def __str__(self): 
        #print Pile properties
        if self.open==False:
            return "Pile has length of "+str(len(self.list_cards))+" Cards and is closed."+ \
                " Its position is at "+str(self.position)
        if self.open==True:
            return "Pile has length of "+str(len(self.list_cards))+" Cards and is open."+ \
                " Its position is at "+str(self.position)
    #giving a card default is top most (last) cards
    def give_card(self,position=-1):
        card=self.list_cards.pop(position)
        return card
    #get card, added at as the last (top most ) always
    def get_card(self,card):
        self.list_cards.append(card)
        #set card position and state to position ans state of pile
        self.list_cards[-1].set_position(self.position)
        self.list_cards[-1].set_state(self.open)
    #recreate pile if nearly empty. parameters: self=pile to be recreated), other pile to be used        
    def refill(self,p_open):
        #refill if there are less than 3 cards
        if len(self.list_cards)<3:
            while len(p_open.list_cards)>1:
                #card closest to the bottom is taken until there is only 1 card
                #cannot use other method in line below, then the top most is taken many times
                card_a=p_open.list_cards.pop(0)
                self.get_card(card_a)
            #shuffle again
            random.shuffle(self.list_cards)
            #return other pile, self in anyway right
            return p_open
    #gets output for machine learning input     
    #gets numpy formated 1D array of 150 length (could be 147 in principle, 3 cards are always elsewhere), value is number when open, closed 20, non existend 30   
    #starts with end because the is top most, most important card
    #all cards besides the first one are ordered by value since the order doe snot matter for 
    #them only the first one can be gottten back
    def get_numeric(self):
        ar=np.zeros((150),int)
        #non existent is default
        ar[:]=30
        
        for i in range(len(self.list_cards)):
            #if open give number
            if self.list_cards[-i-1].open==1:
                ar[i]=self.list_cards[-i-1].number
            #otherwise 20 (not oepnening here,but for player cards)
            else:
                ar[i]=20
        #ordering by value        
        ar[1:150]=np.sort(ar[1:150])        
        return ar         

Now I create new (closed) pile and test that it looks right. 

In [6]:
pile_closed=Pile('create_closed',False)
print(pile_closed)
for i in range(len(pile_closed.list_cards)):
    print(pile_closed.list_cards[i])  
#numeric output for machine learning        
num=pile_closed.get_numeric()
print(num)

Pile has length of 150 Cards and is closed. Its position is at [100, 10]
Card has Value -2 and is closed. Its position is at [100, 10]
Card has Value 10 and is closed. Its position is at [100, 10]
Card has Value -2 and is closed. Its position is at [100, 10]
Card has Value 2 and is closed. Its position is at [100, 10]
Card has Value 6 and is closed. Its position is at [100, 10]
Card has Value 3 and is closed. Its position is at [100, 10]
Card has Value 10 and is closed. Its position is at [100, 10]
Card has Value 8 and is closed. Its position is at [100, 10]
Card has Value 4 and is closed. Its position is at [100, 10]
Card has Value 6 and is closed. Its position is at [100, 10]
Card has Value 5 and is closed. Its position is at [100, 10]
Card has Value 9 and is closed. Its position is at [100, 10]
Card has Value -1 and is closed. Its position is at [100, 10]
Card has Value 0 and is closed. Its position is at [100, 10]
Card has Value 7 and is closed. Its position is at [100, 10]
Card ha

As the next test a make the second (open) pile.

In [7]:
pile_open=Pile('create_open',pile_closed)
print(pile_open)
print(len(pile_open.list_cards),len(pile_closed.list_cards))
#numeric output for machine learning    
num=pile_open.get_numeric()
print(num)
#printing the card properties of the pile
for i in range(len(pile_open.list_cards)):
    print(pile_open.list_cards[i]) 

Pile has length of 1 Cards and is open. Its position is at [300, 10]
1 149
[ 8 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
 30 30 30 30 30 30]
Card has Value 8 and is open. Its position is at [300, 10]


Now I test the giving and getting of cards.

In [8]:
card_a=pile_closed.give_card()
print(card_a)
print(len(pile_closed.list_cards))
print(pile_closed.list_cards[-1])
pile_open.get_card(card_a)
print(len(pile_open.list_cards),pile_open.list_cards[-1])

Card has Value 3 and is closed. Its position is at [100, 10]
148
Card has Value 7 and is closed. Its position is at [100, 10]
2 Card has Value 3 and is open. Its position is at [300, 10]


It works.

As the next test I look on the refill method for the closed pile. 

In [9]:
#length of the pile
print(f"length of closed: {len(pile_closed.list_cards)} length of open: {len(pile_open.list_cards)}")
#now emptyning closed that it can be refilled
while len(pile_closed.list_cards)>2:
    card_a=pile_closed.give_card()
    pile_open.get_card(card_a)
print(f"length of closed: {len(pile_closed.list_cards)} length of open: {len(pile_open.list_cards)}")    
#testing numeric output for machine learning    
num=pile_open.get_numeric()
print("numeric output")
print(num)   
print("Top most card of open is")      
print(pile_open.list_cards[-1])  
#use of refill method
pile_closed.refill(pile_open)
print(f"length of closed: {len(pile_closed.list_cards)} length of open: {len(pile_open.list_cards)}")
print("Top most card of open is")  
print(pile_open.list_cards[-1]) 
print("Top most card of closed is")  
print(pile_closed.list_cards[-1])     

length of closed: 148 length of open: 2
length of closed: 2 length of open: 148
numeric output
[-2 -2 -2 -2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2
  2  3  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  5  5  5
  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7
  7  7  7  8  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9 10
 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 11 11 12 12 12 12 12 12
 12 12 12 12 30 30]
Top most card of open is
Card has Value -2 and is open. Its position is at [300, 10]
length of closed: 149 length of open: 1
Top most card of open is
Card has Value -2 and is open. Its position is at [300, 10]
Top most card of closed is
Card has Value 11 and is closed. Its position is at [100, 10]


Now I create the players class which has the cards. The cards are aranged in a 4 rows of 3 pattern. A Card Position can be empty. I use two lists here for the essential properties, one to check for empty and one which has stored the card. 

In [10]:
#A Card Position can be empty. I use two lists here for the essential properties, one to check for empty and
#one which has stored the card. 
class Player:
    #parameters: string name, mode (human, computer),  level (subkind for mode, more options for computer then human) 
    def __init__(self,name,modes,level,take_from=None):
        #grid positions only for display the game logic uses the list position
        self.positionx=[0,70,140,210,0,70,140,210,0,70,140,210]
        self.positiony=[0,0,0,0,50,50,50,50,100,100,100,100]
        #whether cards exist
        self.exist=[]
        #which cards is there
        self.list_cards=[]
        #name of player
        self.name=name
        #check mode computer (different levels)/human
        self.mode=modes
        #level how random much randomness is there is     
        self.level=level 
        #if take_from is None, only player properties are defined but no cards given
        if take_from!=None:
            for i in range(12):
                #add card which exist
                self.exist.append(1)
                #from pile
                self.list_cards.append(take_from.list_cards.pop(-1))
                #set state to closed
                self.list_cards[-1].set_state(False)
                #set position of card to player grid
                pos=list([self.positionx[i],self.positiony[i]])
                self.list_cards[-1].set_position(pos) 
            #opening 2 cards, currently only open randomly is implemented
            if level<10:
                rang=list(range(12))
                random.shuffle(rang)
                self.list_cards[rang[0]].set_state(True)
                self.list_cards[rang[1]].set_state(True)
        #option for player to do that interactive to be added   
    #for further round in a game only recreate, rest is the same as above in in __init__
    def restart(self,take_from):
        self.exist=[]
        self.list_cards=[]   
        for i in range(12):
            self.exist.append(1)
            self.list_cards.append(take_from.list_cards.pop(-1))
            self.list_cards[-1].set_state(False)
            pos=list([self.positionx[i],self.positiony[i]])
            self.list_cards[-1].set_position(pos) 
        if self.level<10:
            rang=list(range(12))
            random.shuffle(rang)
            self.list_cards[rang[0]].set_state(True)
            self.list_cards[rang[1]].set_state(True) 
    #printing basic player properties        
    def __str__(self):
        return "Player "+self.name+" is a "+self.mode+" in level "+str(self.level)+"."
    #give a card, choosen by index, vanishes: is set to None and not existing
    def give_card(self,i):
        card=self.list_cards[i]
        self.exist[i]=0
        self.list_cards[i]=None
        return card
    #Get a card (parameters: card and index), is always open by design
    def get_card(self,card,i):
        #change ccard
        self.list_cards[i]=card
        #set to existing
        self.exist[i]=1
        #get position of the adress
        pos=list([self.positionx[i],self.positiony[i]])
        #use it
        self.list_cards[i].set_position(pos)
        self.list_cards[i].set_state(True)
    #set_value method only  used in testing    
    #paremeters, index of card in list, number, state     
    def set_value(self,i,number,state):
        #set to existing, open state and desired value
        self.exist[i]=1  
        self.list_cards[i].set_state(True)     
        self.list_cards[i].set_number(number)        
    #method to get list of all closed cards, optional print of how many there are    
    def get_all_closed(self,silent=True):
        #create empty list, append
        closed=[]
        for j in range(12):
            if self.exist[j]==1:
                if self.list_cards[j].open==False:
                    closed.append(j)
        if silent==False:      
            print(self.name+" has "+str(len(closed))+" closed cards")            
        return closed     
    #method to get list of all open cards, optional print of how many there are  
    #same structure as above
    def get_all_open(self,silent=True):   
        open2=[]
        for j in range(12):
            if self.exist[j]==1:
                if self.list_cards[j].open==True:
                    open2.append(j)
        if silent==False:
            print(self.name+" has "+str(len(open2))+" open cards")            
        return open2 
    #method to get list of all existing cards, optional print of how many there are  
    #same structure as above    
    def get_all_cards(self,silent=True):   
        existing=[]
        for j in range(12):
            if self.exist[j]==1:
                existing.append(j)
        if silent==False:
            print(self.name+" has "+str(len(existing))+" cards")            
        return existing    
    #get score at the end of a round, adidng up all number values of existing cards
    def get_score(self):   
        score=0
        for j in range(12):
            if self.exist[j]==1:
                score+=self.list_cards[j].number         
        return score 
    #when 3 open cards in column have the same value, they vanish 
    #check whether cards vanish
    def check_vanish_cards(self):
        #-1 is default when no vanished
        vanish=-1
        #checking the four options
        for j in range(4):
            #3 open of same number
            if self.exist[0+j]==True and self.exist[4+j]==True and self.exist[8+j]==True: 
                if self.list_cards[0+j].open==True and self.list_cards[4+j].open==True and self.list_cards[8+j].open==True and self.list_cards[0+j].number==self.list_cards[4+j].number and self.list_cards[0+j].number==self.list_cards[8+j].number:
                    #is yes index is returned by design there can only one row be added in each turn, thus it is fine
                    #that the methiod cannot handle several
                    vanish=j
        return vanish  
    #method which return vanished when previous method marks them (gets more than -1)
    def vanish_cards(self,row):
        card1=self.list_cards[0+row]
        card2=self.list_cards[4+row]
        card3=self.list_cards[8+row]
        self.exist[0+row]=0
        self.exist[4+row]=0
        self.exist[8+row]=0
        self.list_cards[0+row]=None
        self.list_cards[4+row]=None
        self.list_cards[8+row]=None
        return card1, card2, card3 
    #gets numeric output for machine learning, same principle than for piles
    #gets numpy formated 1D array of 12 length, values is number when open, closed 20, non existedn 30   
    def get_numeric(self):
        ar=np.zeros((12),int)
        for i in range(12):
            if self.exist[i]==0:
                ar[i]=30
            else:
                if self.list_cards[i].open==1:
                    ar[i]=self.list_cards[i].number
                else:
                    ar[i]=20
        return ar   

 I test the player class and methods below. Below I mainly check the vanishing of cards.

In [11]:
#recreate piles to start from normal length
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
#create two players
alpha=Player("alpha",'computer',0,pile_closed)
beta=Player("beta",'computer',0,pile_closed) 
print(alpha,beta)
#numeric output for machine learning        
num2=alpha.get_numeric()
print(num2)
#checking the method which get all cards of a type, needed for the turn function 
closed=alpha.get_all_closed(silent=False)
open2=alpha.get_all_open(silent=False)
exist=beta.get_all_cards(silent=False)
exist2=alpha.get_all_cards(silent=False)
score=alpha.get_score()
print(score)
#checking vanishing method when none need to vanish
check=alpha.check_vanish_cards()
print(check)
#also 4 options where tested
card_1=3
card_2=7
card_3=11
#print values before changing
print(alpha.list_cards[card_1])
print(alpha.list_cards[card_2])
print(alpha.list_cards[card_3])
#setting number to value
alpha.set_value(card_1,5,True)
alpha.set_value(card_2,5,True)
alpha.set_value(card_3,5,True)
#print values
print(alpha.list_cards[card_1])
print(alpha.list_cards[card_2])
print(alpha.list_cards[card_3])
##checking vanishing method when one needs to vanish
check=alpha.check_vanish_cards()
print(check)
#get values
num2=alpha.get_numeric()
print(num2)
#get cards which need to vanish
card1,card2,card3=alpha.vanish_cards(check)
#chekc whether they vanished
print(alpha.exist[card_1])
print(alpha.exist[card_2])
print(alpha.exist[card_3])
print(alpha.list_cards[card_1])
print(alpha.list_cards[card_2])
print(alpha.list_cards[card_3])
#get how cards there are
print(alpha.get_all_cards())
#check cards
print(card1,card2,card3)
#check numeric 
num2=alpha.get_numeric()
print(num2)

Player alpha is a computer in level 0. Player beta is a computer in level 0.
[20 20 20 20 20  3 20 20 20 20  6 20]
alpha has 10 closed cards
alpha has 2 open cards
beta has 12 cards
alpha has 12 cards
83
-1
Card has Value 7 and is closed. Its position is at [210, 0]
Card has Value 10 and is closed. Its position is at [210, 50]
Card has Value -2 and is closed. Its position is at [210, 100]
Card has Value 5 and is open. Its position is at [210, 0]
Card has Value 5 and is open. Its position is at [210, 50]
Card has Value 5 and is open. Its position is at [210, 100]
3
[20 20 20  5 20  3 20  5 20 20  6  5]
0
0
0
None
None
None
[0, 1, 2, 4, 5, 6, 8, 9, 10]
Card has Value 5 and is open. Its position is at [210, 0] Card has Value 5 and is open. Its position is at [210, 50] Card has Value 5 and is open. Its position is at [210, 100]
[20 20 20 30 20  3 20 30 20 20  6 30]


In [12]:
#Here all cards are printed
for i in range(12):
    print(i)
    print(exist[i])
    print(alpha.list_cards[i])

0
0
Card has Value 11 and is closed. Its position is at [0, 0]
1
1
Card has Value 12 and is closed. Its position is at [70, 0]
2
2
Card has Value 3 and is closed. Its position is at [140, 0]
3
3
None
4
4
Card has Value 4 and is closed. Its position is at [0, 50]
5
5
Card has Value 3 and is open. Its position is at [70, 50]
6
6
Card has Value 11 and is closed. Its position is at [140, 50]
7
7
None
8
8
Card has Value 7 and is closed. Its position is at [0, 100]
9
9
Card has Value 11 and is closed. Its position is at [70, 100]
10
10
Card has Value 6 and is open. Its position is at [140, 100]
11
11
None


Below I check the giving and taking of cards. First player to open.

In [13]:
print(f" Card 1 of player: {alpha.list_cards[1]}")
card_a=alpha.give_card(1)
print(card_a)
print(f"Card 1 of player now: {alpha.list_cards[1]}")
print(f"Top most card of open pile: {pile_open.list_cards[-1]}")      
pile_open.get_card(card_a)      
print(f"Top most card of open pile now: {pile_open.list_cards[-1]}")  

 Card 1 of player: Card has Value 12 and is closed. Its position is at [70, 0]
Card has Value 12 and is closed. Its position is at [70, 0]
Card 1 of player now: None
Top most card of open pile: Card has Value 0 and is open. Its position is at [300, 10]
Top most card of open pile now: Card has Value 12 and is open. Its position is at [300, 10]


Now closed pile to player. 

In [15]:
print(f"Top most card of closed pile: {pile_closed.list_cards[-1]}")
print(f"length of closed pile: {len(pile_closed.list_cards)}") 
card_b=pile_closed.give_card()
alpha.get_card(card_b,1)
print(f" Card 1 of player {alpha.list_cards[1]}") 
print(f"Top most card of closed pile now: {pile_closed.list_cards[-1]}")
print(f"length of closed pile now: {len(pile_closed.list_cards)}") 

Top most card of closed pile: Card has Value 6 and is closed. Its position is at [100, 10]
length of closed pile: 125
 Card 1 of player Card has Value 6 and is open. Its position is at [70, 0]
Top most card of closed pile now: Card has Value 4 and is closed. Its position is at [100, 10]
length of closed pile now: 124


Now I built the function which determines which player starts.

In [43]:
#function which dtermines who storts
#parameters: whether is first round of game, player_list, which player ended the last round, 
#,silent whether the result is printed on screen
def who_starts(first,players,last_ender,silent=True):
    #if first round of a game, player with largest sum of open values starts
    if first==True:
        #counter list for all players
        counters=[]
        for i in range(len(players)):
            #counting for each player
            counter=0
            for j in range(12):
                #check when card exists
                if players[i].exist[j]==1:
                    #count when card is open
                    if players[i].list_cards[j].open==1:
                        counter+=players[i].list_cards[j].number
            counters.append(counter)            
        #get maximum
        maxc=max(counters)
        #get index of it, not important whoch chossen when there are several
        starter=players[counters.index(maxc)]
        if silent==False:
            print("Player "+str(starter.name)+" starts")        
        return starter 
    #otherwise the ender of the previous round starts
    else:
        if silent==False:
            print("Player "+str(last_ender.name)+" starts") 
        return last_ender                    

Below it is tested, all startes again to avoid the piles or player are too empty. 

In [45]:
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
alpha=Player("alpha",'computer',0,pile_closed)
beta=Player("beta",'computer',0,pile_closed) 
gamma=Player("gamma",'computer',0,pile_closed) 
delta=Player("delta",'computer',0,pile_closed) 
#different test cases
#players=[alpha,beta,gamma,delta]
players=[alpha,beta,gamma,delta]
#players=[alpha,beta]
#players=[alpha,beta,gamma,delta,epsilon,zeta,eta,theta]
print(players)
for i in range(len(players)):
    print(players[i])
#print values
print("values of all players")
print(alpha.get_numeric())
print(beta.get_numeric())
print(gamma.get_numeric())
print(delta.get_numeric())
#larger value option
a=who_starts(True,players,None,False)
#set starter option
b=who_starts(False,players,beta,False)
#start

[<__main__.Player object at 0x7fb3192fc040>, <__main__.Player object at 0x7fb3192fc310>, <__main__.Player object at 0x7fb3192fc400>, <__main__.Player object at 0x7fb3192fc280>]
Player alpha is a computer in level 0.
Player beta is a computer in level 0.
Player gamma is a computer in level 0.
Player delta is a computer in level 0.
values of all players
[20 20 20 20 20  9  1 20 20 20 20 20]
[20 20 20 20 20 20 20  0 20  1 20 20]
[20 20 20 20 11 20 10 20 20 20 20 20]
[20 20 20 20 20 20 20 20 20  0 20 10]
Player gamma starts
Player beta starts


Function to make a turn, first computer option. Needs later different choices for levels. 

The following output is collected on the turn level optionally for machine learning. First the state before the action, the player id [1], the player state(12), other player state [12*(n-1)], open pile state (150), closed pile length (1), then the actions, boleans for take_open [1] and discarding [1] and the index, the value and state of the choosen player card [3]. The value of closed pile (dummy if closed ignored). That are in total 158+12*N_player



In [123]:
#players is only needed that higher level computer can consider other players card for choices
#possible add past scores for that purpose also
#silent whether results are printed or not
#output whether numeric output is collected for for machine learning or not
#computer mode 1 2 3 tests the pile options
def turn(player,players,pile,discarded,silent=True,output=False):
    #global score(needed when several rounds) and inplay
    global score, in_play
    #numeric output for machine learning
    num=np.zeros((158+12*len(players)),int)
    #which player acting
    for i in range(len(players)):
        if players[i]==player:
            num[0]=i
    #discard and closed pile properties in data collection       
    num[1:151]=discarded.get_numeric()
    num[151]=len(pile.list_cards)   
    num[152:164]=player.get_numeric()
    #get of other players
    player_counter=0
    for i in range(len(players)):
        if players[i]!=player:
            player_counter+=1
            other_player=players[i]
            num[152+player_counter*12:164+player_counter*12]=other_player.get_numeric()
    #choosing now         
    if player.mode=='computer':
        #in level 0 random 50% choice of action
        if player.level==0:
            r_number1=random.random()
            if r_number1<0.5:
                take_open=False
            else:    
                take_open=True
        if player.level==1 or player.level==2:
            take_open=False
        if player.level==3:
            take_open=True            
        num[164+player_counter*12]=take_open   
    #geneneral choose structure does not depend on player    
    if take_open==True:
        #by design
        discard=False
        num[165+player_counter*12]=discard
        if silent==False:
            print("take from open swaps with own")
        #use method for existing cards
        existing=player.get_all_cards()
        #select from them
        #randomly for level 0
        if player.mode=='computer':
            if player.level==0 or player.level==1 or player.level==2 or player.level==3:
                selected=random.choice(existing)
        card_a=player.give_card(selected)
        num[166+player_counter*12]=selected
        num[167+player_counter*12]=card_a.number
        num[168+player_counter*12]=card_a.open
        #dummy for closed pile value
        num[169+player_counter*12]=30
        card_b=pile_open.give_card()
        pile_open.get_card(card_a)
        player.get_card(card_b,selected)
        if silent==False:            
            print("player "+player.name+" has now "+str(player.list_cards[selected].number))
            print("open Pile has now on "+str(pile_open.list_cards[-1].number))  
    #take from close pile        
    else:
        #in computer mode
        if player.mode=='computer':        
            #in level 0 random 50% choice of sub action
            if player.level==0:
                r_number2=random.random()
                if r_number2<0.5:
                    discard=False
                else:    
                    discard=True
            if player.level==1 or player.level==3:
                discard=False
            if player.level==2:
                discard=True                
        num[164+player_counter*12]=take_open     
        num[165+player_counter*12]=discard 
        #taking does not depend on mode
        if discard==True:
            if silent==False:                
                print("take from closed discards it")
            #put card on open pile from closed
            card_a=pile_closed.give_card()
            num[169+player_counter*12]=card_a.number 
            pile_open.get_card(card_a)
            #open random closed cards using method
            closed=player.get_all_closed()
            if player.mode=='computer':  
            #randomly for level 0                
                if player.level==0 or player.level==1  or player.level==2 or player.level==3:
                    selected=random.choice(closed)  
            num[166+player_counter*12]=selected
            num[167+player_counter*12]=player.list_cards[selected].number
            num[168+player_counter*12]=player.list_cards[selected].open    
            player.list_cards[selected].set_state(True)
            if silent==False:                

                print("open Pile has now on top "+str(pile_open.list_cards[-1].number))     
        #next close card not discard mode (last mode)  
        else:
            if silent==False:
                print("take from closed swaps with own")
            #choose from existing cards
            existing=player.get_all_cards()
            if player.mode=='computer':              
                #randomly for level 0
                if player.level==0 or player.level==1 or player.level==2 or player.level==3:
                    selected=random.choice(existing)
            card_a=player.give_card(selected)
            num[166+player_counter*12]=selected
            num[167+player_counter*12]=card_a.number
            num[168+player_counter*12]=card_a.open
            card_b=pile_closed.give_card()
            num[169+player_counter*12]=card_b.number 
            pile_open.get_card(card_a)
            player.get_card(card_b,selected)
            if silent==False:                
                print("player "+player.name+" has now "+str(player.list_cards[selected].number))                
                print("open Pile has now in top "+str(pile_open.list_cards[-1].number))
    #check for vanishing cards
    van=player.check_vanish_cards()
    if van>-1:
        card1,card2,card3=player.vanish_cards(van)
        pile_open.get_card(card1)
        pile_open.get_card(card2)
        pile_open.get_card(card3)
        if silent==False:        
            print("3 "+str(card1.number)+" vanish from Player")
    #refill closed pile if needed 
    pile_closed.refill(pile_open)
    #check whether there are still closed cards 
    closed=player.get_all_closed()
    if len(closed)==0:
        in_play=False 
        if silent==False:        
            print("player "+player.name+" opened all cards")
    if output==False:
        return player, players, pile, discarded  
    else:
        return player, players, pile, discarded, num

Playing some turns, it seems to work. 

In [125]:
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
alpha=Player("alpha",'computer',3,pile_closed)
beta=Player("beta",'computer',3,pile_closed) 
players=[alpha,beta]
#always smae player here
#not silent and output collection
player,players, pile,discarded,num=turn(alpha,players,pile_closed,pile_open,False,True)
print(len(alpha.get_all_closed()))
print("numeric output is")
print(num)
print(num[152:164])
print(num[164:176])
#with output collection
player,players, pile,discarded,num=turn(alpha,players,pile_closed,pile_open,False,True)
print(len(alpha.get_all_closed()))
print("numeric output is")
print(num)
print(num[152:164])
print(num[164:176])
player,players, pile,discarded,num=turn(alpha,players,pile_closed,pile_open,False,True)
print(len(alpha.get_all_closed()))
print("numeric output is")
print(num)
print(num[152:164])
print(num[164:176])

take from open swaps with own
player alpha has now 1
open Pile has now on 3
9
numeric output is
[  0   1  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30
  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30
  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30
  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30
  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30
  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30
  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30
  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30  30
  30  30  30  30  30  30  30 125  20  20  20  20  20  20   9  10  20  20
  20  20  20  20  20  20  20  20  20   7  20  20  12  20   1   0   9   3
   0  30]
[20 20 20 20 20 20  9 10 20 20 20 20]
[20 20 20 20 20 20 20  7 20 20 12 20]
take from open swaps with own
player alpha has now 3
open Pile has now on 3
8
numeric ou

Now creating function for round. 

In [243]:
#levels (modes) defined for computer
comp_level_list = [0,1,2,3]

In [244]:
def allowed_modes(names,nature,levels):
    global comp_level_list
    #allowed testing should be outsourced in function 
    allowed=True
    if len(names)<2 or len(names)>8 or len(names)!=len(nature) or len(names)!=len(levels):
        allowed=False
    else:
        for i in range(len(names)):
            #human is not implemented and will have different levels
            if nature[i]=='computer' and any(levels[i] in comp_level_list for item in comp_level_list)==False:
                allowed=False
            #for now because human not implemented    
            if nature[i]!='computer':
                allowed=False
    return allowed             

In [256]:
#parameters player names list, mode list, level, list, pause per turn (seconds), silent printing
#output collection optional (for machine learning)
def skyjo_round(names,nature,levels,pause,first_round,silent=True,output=False):
    allowed=allowed_modes(names,nature,levels)
    #report and abort when not defined            
    if allowed==False:
        print("The player input \nnames:"+str(names)+"\nnature:"+str(nature)+"\nlevel:"+str(levels))
        print("is not allowed.")
    else:
        global in_play, players, pile_open, pile_closed
        #create piles
        pile_closed=Pile('create_closed',False)
        pile_open=Pile('create_open',pile_closed)
        #for output collection first a list which is at the end converted to array
        outputs=[]
        if first_round==True:
            #create players and add to list and in first round 
            players=[]
            for i in range(len(names)):
                players.append(Player(names[i],nature[i],levels[i]))
        for i in range(len(names)):
            #fill them always
            players[i].restart(pile_closed)            
        if silent==False:
            print("closed pile has "+str(len(pile_closed.list_cards))+" cards")        
        #drawing here?    does not work 
        #frame.set_draw_handler(draw)
        #determine start player
        if first_round==True:
            if silent==False:
                print("game starts")
            for i in range(len(players)):
                if silent==False:
                    print(players[i])
            starter=who_starts(True,players,None)
        else: 
            starter=who_starts(False,players,first_round)
            if silent==False:
                print("new round starts")
        if silent==False: 
            print("Player "+starter.name+" starts")
        #index of starter
        idx=players.index(starter)
        if pause>0:
            #slow round to be able to observe in GUI
            time.sleep(pause)
        #first turn
        if output==False:
            turn(starter,players,pile_closed,pile_open,silent,False)
        else:
            starter,players,pile,discarded,num=turn(starter,players,pile_closed,pile_open,silent,True)
            outputs.append(num)
        if silent==False:
            print("closed pile has "+str(len(pile_closed.list_cards))+" cards")     
        #counter set to index of starter
        counter=players.index(starter)
        in_play=True
        finisher=0
        while in_play==True:
            if pause>0:
                time.sleep(pause)
            #now alternating turn while in play
            counter+=1
            player=players[counter%len(players)]
            if silent==False:
                print("next turn is of "+player.name)            
            if output==False:           
                player,players,pile_closed,pile_open=turn(player,players,pile_closed,pile_open,silent,False)
            else:
                player,players, pile,discarded,num=turn(player,players,pile_closed,pile_open,silent,True)   
                outputs.append(num)            
            if silent==False:
                print(player.name)
                print("closed pile has "+str(len(pile_closed.list_cards))+" cards")        
            finisher=counter%len(players)
        
        #one turn of the other players at the  end
        last_player=None
        if in_play==False:
            for i in range(len(players)-1):
                if pause>0:
                    time.sleep(pause)
                counter+=1
                player=players[counter%len(players)]
                #will start next round in a game
                last_player=player 
                if silent==False:
                    print("next turn is of "+player.name)
                if output==False:
                    player,players,pile_closed,pile_open=turn(player,players,pile_closed,pile_open,silent,False)
                else:
                    player,players, pile,discarded,num=turn(player,players,pile_closed,pile_open,silent,True)    
                    outputs.append(num)             
                if silent==False:
                    print("closed pile has "+str(len(pile_closed.list_cards))+" cards")
        if output==True:
            if silent==False:
                print(str(counter-idx+1)+" turns were made")     
            num2=np.zeros((len(num),len(outputs)),int)
            for i in range(len(outputs)):
                num2[:,i]=outputs[i]  
        scores=[]
        for i in range(len(players)):
            scores.append(players[i].get_score())   
        #need to consider who finished the round here
        #if not the unique lowest (and positive) then score is doubled
        #controll parameter to avoid several doublings
        not_finisher=False
        for i in range(len(players)):       
            if i!=finisher and not_finisher==False and min(scores)>0:
                if scores[finisher]>=scores[i] and scores[finisher]>0:
                    scores[finisher]*=2
                    not_finisher=True
        if silent==False:
            print("score of round is "+str(scores))  
        if output==False:    
            return scores, counter-idx+1, last_player
        else:    
            return scores, counter-idx+1, last_player, num2    

Testing round with output.

In [259]:
#test round # seems to works but less clear why it works and why the number of globals and returns are needed
#creates names and properties of player
names=['alpha','beta','gamma']
nature=['computer','computer','computer']
levels=[2,2,2]
nn=int(1)
start_time=time.time()
result=np.zeros((len(names)+2,nn))
for i in range(nn):
    scores,turns,last_player,num=skyjo_round(names,nature,levels,0,True,False,True)
    print(scores)
    for j in range(len(names)):
        result[j,i]=scores[j]
    result[len(names),i]=turns
    print(num.shape)
#counter now fine, refill now also implemented (rarely needed for 2 player but will be important for more)
stop_time=time.time()
print(stop_time-start_time)
print(result)

closed pile has 113 cards
game starts
Player alpha is a computer in level 2.
Player beta is a computer in level 2.
Player gamma is a computer in level 2.
Player beta starts
take from closed discards it
open Pile has now on top 11
closed pile has 112 cards
next turn is of gamma
take from closed discards it
open Pile has now on top 8
gamma
closed pile has 111 cards
next turn is of alpha
take from closed discards it
open Pile has now on top 3
alpha
closed pile has 110 cards
next turn is of beta
take from closed discards it
open Pile has now on top 6
beta
closed pile has 109 cards
next turn is of gamma
take from closed discards it
open Pile has now on top 7
gamma
closed pile has 108 cards
next turn is of alpha
take from closed discards it
open Pile has now on top 8
alpha
closed pile has 107 cards
next turn is of beta
take from closed discards it
open Pile has now on top 11
beta
closed pile has 106 cards
next turn is of gamma
take from closed discards it
open Pile has now on top 3
gamma
clo

Now many rounds silent to see whether it terminates sometimes. 

In [133]:
#test round # seems to works but less clear why it works and why the number of globals and returns are needed
#creates names and properties of player
names=['alpha','beta','gamma','delta']
nature=['computer','computer','computer','computer']
levels=[1,1,1,1]
nn=int(100)
result=np.zeros((len(names)+2,nn))
start_time=time.time()
for i in range(nn):
    print(f"run number {i}")
    scores,turns,last_player=skyjo_round(names,nature,levels,0,True,True,False) 
    for j in range(len(names)):
        result[j,i]=scores[j]
    result[len(names),i]=turns
#counter now fine
stop_time=time.time()
print(stop_time-start_time)
print(min(result[2]),np.mean(result[2]),max(result[2]))

run number 0
run number 1
run number 2
run number 3
run number 4
run number 5
run number 6
run number 7
run number 8
run number 9
run number 10
run number 11
run number 12
run number 13
run number 14
run number 15
run number 16
run number 17
run number 18
run number 19
run number 20
run number 21
run number 22
run number 23
run number 24
run number 25
run number 26
run number 27
run number 28
run number 29
run number 30
run number 31
run number 32
run number 33
run number 34
run number 35
run number 36
run number 37
run number 38
run number 39
run number 40
run number 41
run number 42
run number 43
run number 44
run number 45
run number 46
run number 47
run number 48
run number 49
run number 50
run number 51
run number 52
run number 53
run number 54
run number 55
run number 56
run number 57
run number 58
run number 59
run number 60
run number 61
run number 62
run number 63
run number 64
run number 65
run number 66
run number 67
run number 68
run number 69
run number 70
run number 71
ru

Now I add the function to play full game which can consist of several rounds. A game ends when one player has 100 points of more.


In [313]:
#parameters, names, nature and levels of players (lists of same length), pause (second can be zero), 
#silent (but not yet implemented), output (not yet implemented)
def skyjo_game(names,nature,levels,pause,silent=True,output=False):
    #if check whether parameters are allowed *defined)
    #globally defoined levels
    allowed=allowed_modes(names,nature,levels)
    #report and abort when not defined            
    if allowed==False:
        print("The player input \nnames:"+str(names)+"\nnature:"+str(nature)+"\nlevel:"+str(levels))
        print("is not allowed.")
    else:
        #play the game
        #total book keeping
        round_counter=0
        #total score is list of player length
        tot_score=[]
        for i in range(len(names)):
            tot_score.append(0)
        #marker for game ove   
        over=False
        #create players and add to list (probably needs mode were name is not recreated for later round in game)
        players=[]
        #for output collection first a list which is at the end converted to array
        for i in range(len(names)):
            players.append(Player(names[i],nature[i],levels[i]))
        if output==False:
            scores,turns,last_player=skyjo_round(names,nature,levels,pause,True,True,output) 
        else:
            #output and scores collection
            outputs=[]
            scores=[]            
            score,turns,last_player,num=skyjo_round(names,nature,levels,pause,True,True,output)   
            outputs.append(num)
            scores.append(score)
    
        if silent==False:
            print("New Skyjo game")
            print("first round")
            print("score is "+str(score))
        
        for i in range(len(tot_score)):
            tot_score[i]+=score[i]
        if max(tot_score)>=100:
            over=True
        else:
            #more rounds until first player has at least 100
            while max(tot_score)<100:
                round_counter+=1
                if silent==False:
                    print("playing round "+str(round_counter+1))
                if output==False:
                    score,turns,last_player=skyjo_round(names,nature,levels,pause,last_player,True,output)
                else: 
                    score,turns,last_player,num=skyjo_round(names,nature,levels,pause,last_player,True,output)
                    outputs.append(num)
                    scores.append(score)
                for i in range(len(tot_score)):
                    tot_score[i]+=score[i] 
                if silent==False:
                   print("current score is "+str(tot_score))   
            else:
                over=True
        if over==True:
            if silent==False:
                print("game is over")   
            winner=[]    
            for i in range(len(players)):
                if tot_score[i]==min(tot_score):
                    winner.append(1)
                    if silent==False:
                        print("Player "+players[i].name+" won")    
                else:
                    winner.append(0)
            if output==True:
                #combine outputs in one array 
                #figure out size of it 
                row_counter=0
                for i in range(round_counter+1):
                    row_counter+=outputs[i].shape[1]
                #include besides propagte of round also rpund number, turn number, scores and final winner array
                final=np.zeros((outputs[0].shape[0]+4+len(players),row_counter))
                row_counter=0
                for i in range(round_counter+1):
                    final[0:outputs[0].shape[0],row_counter:row_counter+outputs[i].shape[1]]=outputs[i]
                    for j in range(outputs[i].shape[1]):
                        final[outputs[0].shape[0],row_counter+j]=i
                        final[outputs[0].shape[0]+1,row_counter+j]=j                        
                        final[outputs[0].shape[0]+2:outputs[0].shape[0]+4,row_counter+j]=scores[i]
                        final[outputs[0].shape[0]+4:final.shape[0],row_counter+j]=winner
                    row_counter+=outputs[i].shape[1] 
            return final, winner       

In [199]:
level_list = [0,1,2,3]

In [314]:
#problems with 3 or more players, often too high score for one
names=['alpha','beta']
nature=['computer','computer']
levels=[3,1]
print(names,nature,levels)


['alpha', 'beta'] ['computer', 'computer'] [3, 1]


In [318]:
final,winner=skyjo_game(names,nature,levels,0,False,True)

New Skyjo game
first round
score is [43, 64]
playing round 2
current score is [183, 116]
game is over
Player beta won


In [319]:
print(winner)
print(final[:,0])
print(final[:,60])

[0, 1]
[  0.   4.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30. 125.  20.  20.
  20.  20.  20.  20.  20.  20.  20.  20.  11.  11.  20.  20.  -1.  20.
  20.  20.  11.  20.  20.  20.  20.  20.   1.   0.   9.   5.   0.  30.
   0.   0.  43.  64.   0.   1.]
[  1.   8.   7.   5.  -1.  -1.   7.  1

Outputs works, the question is whether the computer trains on rounds or games. 

The game function seems to work, but the round (or below) in it has likely still minor problems. 

Function to draw the results in the GUI  screen.  Should be disactivated for computer only.

In [ ]:
# draw function
def draw(canvas):
    global pile_open,pile_closed, players
    #display the top most card
    p_open=pile_open.list_cards[-1]
    p_closed=pile_closed.list_cards[-1]
    # test to make sure that card.draw works
    p_open.draw(canvas)
    p_closed.draw(canvas)
    for i in range(len(players)):
        for j in range(12):
            #only cards which exist are drawn:
            if players[i].exist[j]==1:
                card=players[i].list_cards[j]
                if len(players)==2:
                    drawpos=list(card.position)
                    drawpos[0]=i*290+players[i].positionx[j]
                    drawpos[1]=100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%2)*290,180*(1+(i//2))+90),15,'Black')
                if abs(len(players)-3.5)==0.5:
                    drawpos=list(card.position)
                    drawpos[0]=(i%2)*290+players[i].positionx[j]
                    drawpos[1]=180*(i//2)+100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%2)*290,180*(1+(i//2))+90),15,'Black')
                if abs(len(players)-5.5)==0.5:
                    drawpos=list(card.position)
                    drawpos[0]=(i%3)*290+players[i].positionx[j]
                    drawpos[1]=180*(i//3)+100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%3)*290,180*(1+(i//3))+90),15,'Black')     
                if abs(len(players)-7.5)==0.5:
                    drawpos=list(card.position)
                    drawpos[0]=(i%4)*290+players[i].positionx[j]
                    drawpos[1]=180*(i//4)+100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%4)*290,180*(1+(i//4))+90),15,'Black')    

Drawing works now for all allowed player numbers. 

In [ ]:
def hit_start():
    global start
    #better but still not really
    players,scores,turns=skyjo_round(names,nature,levels,1) 
    #only appears at the end

In [ ]:
#seems not work well in the notebook sometimes, not clear whether real problem
#likely ignored since notebook is not so important here
names=['alpha','beta']
nature=['computer','computer']
levels=[0,0]
if len(names)==2:
    frame = simplegui.create_frame("Skyjo", 290+280*(1), 100+3*50+30) 
if abs(len(names)-3.5)==0.5:
    frame = simplegui.create_frame("Skyjo", 290+280*(1), 100+3*50*2+60)
if abs(len(names)-5.5)==0.5:
    frame = simplegui.create_frame("Skyjo", 290+280*(2)+10, 100+3*50*2+60)    
if abs(len(names)-7.5)==0.5:
    frame = simplegui.create_frame("Skyjo", 290+280*(3)+20, 100+3*50*2+60)  
frame.set_canvas_background("White")
frame.add_button("start", hit_start, 200)    
frame.set_draw_handler(draw)
frame.start()
players,scores,turns=skyjo_round(names,nature,levels,1) 
#drawing of comuter does not work 

In [ ]:
exit()
